## 1) 传统 NLP 的“核心矛盾”到底是什么？
传统 NLP 大多数方法都在解决同一件事：
> 如何把离散符号（词/字）变成可学习的数值表示，并在数据有限、计算受限、语义弱监督的情况下完成任务。

所以传统 NLP 的主线就是两层：
1. 表示层（Feature / Representation）:BoW/TF-IDF/BM25、n-gram、LSA、LDA、word2vec/fastText/doc2vec …

2. 模型层（Model）:朴素贝叶斯、逻辑回归、线性 SVM、HMM/CRF …

## 2）一句话区分 TF-IDF、BM25、Embedding
- TF-IDF：在“词袋”假设下，用“词频 × 逆文档频率”强调区分性词。

- BM25：IR（信息检索）里更强的 TF-IDF 变体，额外做了长度归一和TF 饱和，更贴近检索打分。

- Embedding（word2vec 等）：用预测任务（上下文/共现）学到稠密语义空间，相似词更近。

## 3）为什么不用 TF-IDF？

（1）承认 TF-IDF 的价值：
TF-IDF 作为 baseline 非常强，训练快、可解释、对小数据/线性模型友好。

（2）说明不用它的“本质原因”：表示能力上限太低
TF-IDF 的核心缺陷来自词袋假设与稀疏高维：

不建模语义：同义词“好/不错/优秀”完全是不同维度 → 泛化差

不建模上下文：多义词“苹果（水果/公司）”无法区分

弱序信息：除非上 n-gram，否则“我打你/你打我”几乎一样

稀疏高维：维度≈词表大小，长尾词多，OOV/拼写变体很麻烦

跨域迁移弱：词分布换了（领域/风格），权重就飘

（3）给出替代方案 + 选择依据：

语义匹配/问答/召回：更倾向 embedding（word2vec/fastText/句向量）或更现代的预训练模型

可解释 + 数据小 + 业务需要看词权重：TF-IDF 仍然非常合适

检索排序：BM25 往往比 TF-IDF 更稳

## 4) 对比总结表

**文本表示方法对比表（传统 NLP 核心）**

| 方法                | 向量形态      | 是否稀疏 |  词序 | 语义能力 | 可解释性 | 数据需求 | 优点                | 缺点             | 典型场景             |
| ----------------- | --------- | ---: | --: | ---: | ---: | ---: | ----------------- | -------------- | ---------------- |
| One-hot / BoW     | 词表维度计数    |    ✅ |   ❌ |    ❌ |   ✅✅ |    低 | 简单强基线             | 同义/多义/上下文全丢    | 小数据分类 baseline   |
| TF-IDF            | BoW + 词权重 |    ✅ |   ❌ |    ❌ |   ✅✅ |    低 | 强区分词、线性可用         | 语义弱、稀疏高维       | 文本分类/聚类 baseline |
| BM25              | 词项匹配打分    |    - |   ❌ |    ❌ |    ✅ |    低 | 长度归一 + TF 饱和，检索强  | 仍是词匹配，无语义      | 传统检索召回/排序        |
| n-gram（word/char） | 扩展词袋      |    ✅ | 部分✅ |    ❌ |    ✅ |    中 | 捕捉局部短语/拼写         | 维度爆炸/稀疏更重      | 短文本、拼写鲁棒、工业常用    |
| Hashing Trick     | 哈希到固定维度   |    ✅ |   ❌ |    ❌ |    ❌ |    低 | 内存友好、可流式          | 冲突不可解释         | 大规模线上特征          |
| LSA/LSI（SVD）      | 低维线性语义    |    ❌ |   ❌ |   弱✅ |   弱✅ |    中 | 降维、抗噪             | 线性假设、主题混叠      | 相似度/聚类/检索增强      |
| LDA（主题模型）         | 文档-主题分布   |    ❌ |   ❌ |   弱✅ |    ✅ |  中-高 | 可解释主题、无监督         | 主题不一定对应语义，调参麻烦 | 主题发现/用户画像        |
| word2vec（CBOW/SG） | 词向量       |    ❌ | 间接✅ |    ✅ |   弱✅ |    高 | 捕捉同义/类比关系         | 无上下文（静态词向量）    | 语义特征、相似词、传统匹配    |
| GloVe             | 词向量       |    ❌ | 间接✅ |    ✅ |   弱✅ |    高 | 基于全局共现统计          | 同样是静态词向量       | 同上               |
| fastText          | 子词词向量     |    ❌ | 间接✅ |    ✅ |   弱✅ |    高 | OOV/拼写变体更稳        | 仍无上下文          | 多语言/噪声文本         |
| doc2vec           | 文档向量      |    ❌ | 间接✅ | ✅/弱✅ |   弱✅ |    高 | 直接得 doc embedding | 训练不稳、依赖语料      | 文档检索/聚类          |


**统模型方法对比表**

| 模型        | 判别/生成 | 线性/非线性 | 对特征要求        | 优点                | 缺点        | 典型任务          |
| --------- | ----- | ------ | ------------ | ----------------- | --------- | ------------- |
| 朴素贝叶斯（NB） | 生成式   | 线性     | 稀疏 BoW 很友好   | 极快、数据少也能跑         | 条件独立假设很强  | 文本分类 baseline |
| 逻辑回归（LR）  | 判别式   | 线性     | TF-IDF/稀疏特征强 | 稳、可解释、好调参         | 表示不够就到顶   | 分类/多标签        |
| 线性 SVM    | 判别式   | 线性     | 高维稀疏很强       | margin 大、效果常优于 LR | 概率输出需校准   | 分类、情感         |
| kNN       | 非参数   | 非线性    | 距离度量敏感       | 简单直观              | 预测慢、维度灾难  | 小规模检索/分类      |
| HMM       | 生成式   | 序列模型   | 需马尔可夫假设      | 结构清晰              | 特征表达弱     | 分词/词性（早期）     |
| CRF（线性链）  | 判别式   | 序列模型   | 可塞大量手工特征     | 序列标注强基线           | 特征工程重、训练慢 | NER/分词/slot   |

## QA

**任务类型是什么？**
分类 / 序列标注 / 检索召回 / 相似度 / 主题发现

语义与上下文是不是核心？

是：TF-IDF 很可能到顶（尤其语义匹配/同义改写/问答）

否：TF-IDF + 线性模型常常非常强

数据规模与可解释要求？

数据小、要解释：TF-IDF/LR/SVM

数据大、要泛化：embedding/更现代方法（哪怕传统也优先 fastText/LSA）

**为什么不用 TF-IDF**

TF-IDF 我一定会先做 baseline，因为它快、可解释，对小数据和线性模型效果很稳。但如果任务核心是语义匹配或需要上下文区分，比如同义词、改写、问答检索，TF-IDF 的词袋假设决定了它缺少语义与上下文建模能力：同义词维度不同、多义词无法消歧，长文本还会被稀疏高维影响泛化。此时我会优先用 BM25 做检索基线，再用 embedding（word2vec/fastText/句向量）或更强的上下文表示来提升语义召回与鲁棒性。是否使用 TF-IDF 取决于任务是否需要语义和上下文，以及可解释与数据规模约束。